In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor

/Users/mariofilho/miniconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
data = pd.read_feather("train.f")

# Prequential Sliding

In [3]:
data['block'] = np.trunc(data['era']*.1).astype(int)
data.loc[data['block'] == 12, 'block'] = 11

In [4]:
data['block'].value_counts().sort_index()

0     24515
1     34600
2     37444
3     41101
4     43439
5     48186
6     46831
7     40403
8     43971
9     45609
10    46107
11    49602
Name: block, dtype: int64

In [5]:
results_val = []

for block in range(1,12):
    print("Train block {} - Validation Block {}".format(block - 1, block))
    

    train = data[data['block'] == block-1]
    val = data[data['block'] == block]
    
    X_train = train.filter(regex=r'feature')
    X_val = val.filter(regex=r'feature')

    y_train = train['target_kazutsugi']
    y_val = val['target_kazutsugi']
     

    mdl = LGBMRegressor(max_depth=5, num_leaves=2**5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, random_state=0)
    mdl.fit(X_train, y_train)
    
    predictions = pd.Series(mdl.predict(X_val))
    ranked_predictions = predictions.rank(pct=True, method="first")
    correlation = np.corrcoef(y_val, ranked_predictions)[0, 1]
    #print(correlation)
    
    results_val.append(correlation)
    print("Correlation {}".format(correlation))
    print()
   

Train block 0 - Validation Block 1
Correlation 0.04237943213599219

Train block 1 - Validation Block 2
Correlation 0.044973582272642934

Train block 2 - Validation Block 3
Correlation 0.04851843964131706

Train block 3 - Validation Block 4
Correlation 0.03599879518646091

Train block 4 - Validation Block 5
Correlation 0.016612730692345163

Train block 5 - Validation Block 6
Correlation 0.019732954106230282

Train block 6 - Validation Block 7
Correlation 0.025411149394669253

Train block 7 - Validation Block 8
Correlation 0.035044580906317006

Train block 8 - Validation Block 9
Correlation 0.038476835565653685

Train block 9 - Validation Block 10
Correlation 0.02896392251489324

Train block 10 - Validation Block 11
Correlation 0.0339355345487348



In [6]:
np.median(results_val)

0.035044580906317006

In [7]:
np.min(results_val)

0.016612730692345163

In [8]:
np.max(results_val)

0.04851843964131706

In [9]:
np.mean(results_val)

0.03364072336047786

In [10]:
len(results_val)

11

# Prequential Sliding with Gap
- gap original - remover apenas era mais próxima do treino

In [11]:
results_val = []

for block in range(2,12):
    print("Train block {} - Gap Block {} - Validation Block {}".format(block - 2, block - 1,  block))

    train = data[data['block'] == block-2]
    val = data[data['block'] == block]
    
    X_train = train.filter(regex=r'feature')
    X_val = val.filter(regex=r'feature')

    y_train = train['target_kazutsugi']
    y_val = val['target_kazutsugi']
     

    mdl = LGBMRegressor(max_depth=5, num_leaves=2**5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, random_state=0)
    mdl.fit(X_train, y_train)
    
    predictions = pd.Series(mdl.predict(X_val))
    ranked_predictions = predictions.rank(pct=True, method="first")
    correlation = np.corrcoef(y_val, ranked_predictions)[0, 1]
    #print(correlation)
    
    results_val.append(correlation)
    print("Correlation {}".format(correlation))
    print()
   

Train block 0 - Gap Block 1 - Validation Block 2
Correlation 0.05345227304313937

Train block 1 - Gap Block 2 - Validation Block 3
Correlation 0.04509813993002888

Train block 2 - Gap Block 3 - Validation Block 4
Correlation 0.039443046756817826

Train block 3 - Gap Block 4 - Validation Block 5
Correlation 0.023738960188158055

Train block 4 - Gap Block 5 - Validation Block 6
Correlation 0.017413657800124795

Train block 5 - Gap Block 6 - Validation Block 7
Correlation 0.024640167304435363

Train block 6 - Gap Block 7 - Validation Block 8
Correlation 0.012308896744822391

Train block 7 - Gap Block 8 - Validation Block 9
Correlation 0.0385174953842173

Train block 8 - Gap Block 9 - Validation Block 10
Correlation 0.02333269112437655

Train block 9 - Gap Block 10 - Validation Block 11
Correlation 0.03035357473693364



In [17]:
np.median(results_val)

0.027496871020684502

In [18]:
np.min(results_val)

0.012308896744822391

In [19]:
np.max(results_val)

0.05345227304313937

In [20]:
np.mean(results_val)

0.030829890301305413

In [21]:
len(results_val)

10